In [2]:
import cv2
import os
import glob
import shutil
import numpy as np

# Input folders
input_folders = ['no', 'yes']

# Output folder for resized images
output_folder = 'resized_images'
# Remove existing output folder and create a new one
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)
os.makedirs(output_folder)

# Resize images, apply sharpening filter, and save them to output folder
for folder in input_folders:
    for img_path in glob.glob(os.path.join(folder, "*.png")):
        image = cv2.imread(img_path)
        img_resized = cv2.resize(image, (224, 224))

        # Convert to RGB
        img_resized_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)

        # Apply sharpening filter
        kernel = np.array([[-1,-1,-1],
                           [-1, 9,-1],
                           [-1,-1,-1]], dtype=np.float32) # Sharpening kernel
        img_sharpened = cv2.filter2D(img_resized_rgb, -1, kernel)

        # Determine label based on filename
        if img_path.endswith("class0.png"):
            label = 0
        elif img_path.endswith("class1.png"):
            label = 1
        else:
            label = None  # Handle other cases here

        # Save the resized and sharpened RGB image to the output folder
        filename = f"{label}_{os.path.basename(img_path)}"
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, cv2.cvtColor(img_sharpened, cv2.COLOR_RGB2BGR))

        

In [4]:
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

# Define a function to load and preprocess images
def load_image(file_path):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = img / 255.0  # Normalize pixel values to [0, 1]
    return img

# Load resized images and their labels
image_paths = glob.glob(os.path.join(output_folder, "*.png"))
labels = [int(os.path.basename(img_path).split('_')[0]) for img_path in image_paths]
images = [load_image(img_path) for img_path in image_paths]

# Convert lists to TensorFlow tensors
images = tf.convert_to_tensor(images)
labels = tf.convert_to_tensor(labels)

# Define your CNN model using TensorFlow/Keras
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification, so using sigmoid activation
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Define k-fold cross-validation
k = 2
skf = StratifiedKFold(n_splits=k)

# Perform k-fold cross-validation
acc_scores = []
for train_index, test_index in skf.split(images, labels):
    X_train, X_test = images[train_index], images[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    # Train the model
    model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=1)

    # Evaluate the model
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    acc_scores.append(accuracy)

# Calculate mean accuracy
mean_accuracy = sum(acc_scores) / len(acc_scores)
print("Mean Accuracy:", mean_accuracy)


c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(
